In [74]:
import sys
sys.path.append('/home/sensetime/ws/common_math/algorithm/idm/build')
from idm_case_ploter import IDMCasePloter
from idm_case_manager import  IDMCaseManager
import pybind_idm
from bokeh.plotting import figure, show, output_notebook
from idm_debug import IDMDebug
from bokeh.plotting import figure, show, output_file, output_notebook
import argparse
from scipy.interpolate import interp1d
import numpy as np
from idm_case_manager import IDMCaseManager
from idm_case_ploter import IDMCasePloter
import sys

def linear_interpolation(x, y, x_new):
    f = interp1d(x, y, kind='linear')
    return f(x_new)

case_path = '/idm_case_file'
case_manager = IDMCaseManager(case_path)
picked_case = case_manager.get_case('follow_40kph')
# print(picked_case)

t_triger_std = picked_case['idm_info']['t_triger']
# read leader info
leadercarlength_std = picked_case['idm_info']['leader_length']

s_leader_init = picked_case['idm_info']['s_leader_init']

v_leader_init = picked_case['idm_info']['v_leader_init']

a_leader_init = picked_case['idm_info']['a_leader_init']

exist_leader = picked_case['idm_info']['exist_leader']

# read v_refs
v_refs = picked_case['idm_info']['v_refs']


# print(v_refs,exist_leader, s_leader_init, v_leader_init, a_leader_init)

# read ego init info
ego_s = picked_case['ego_init_info']['s']
ego_v = picked_case['ego_init_info']['v']
ego_a = picked_case['ego_init_info']['a']

# print(ego_s,ego_v,ego_a)


# linear_interpolation info to idm input can use
leader_info = []

t_triger_list = []
leadercarlength_list = []
s_leader_list = []
v_leader_list = []

exist_leader_list = []
v_refs_list = []

print(s_leader_list,v_leader_list)
for i in np.arange(0, 6.1, 0.2):
        t_triger_list.append(i)
for t in t_triger_list:
        exist_leader_list.append(exist_leader)
        leadercarlength_list.append(leadercarlength_std)
        v_refs_list.append(v_refs)
# print("exist_leader_list:",len(exist_leader_list),exist_leader_list)
# print("leadercarlength_list:",len(leadercarlength_list),leadercarlength_list)
# print("v_refs_list:",len(v_refs_list),v_refs_list)

tmp_leader_s = s_leader_init
tmp_leader_v = v_leader_init
const_a = a_leader_init

for t in t_triger_list:
        v_leader_list.append(tmp_leader_v)
        s_leader_list.append(tmp_leader_s)
        tmp_leader_v += const_a * 0.2
        tmp_leader_s += tmp_leader_v * 0.2 + 0.5 * const_a * 0.2 * 0.2

print("s leadre info:",len(s_leader_list),s_leader_list)
print("v leadre info:",len(v_leader_list),v_leader_list)



****************************** IDMCaseManager start read case ******************************
{'idm_info': {'a_leader_init': 1, 'leader_length': 4.0, 't_triger': [0.0, 6.0], 's_leader_init': 17, 'v_refs': 22.22, 'v_leader_init': 15.8, 'exist_leader': 1}, 'ego_init_info': {'s': -0.5, 'v': 18.08, 'a': 0}}
[] []
s leadre info: 31 [17, 20.22, 23.48, 26.78, 30.12, 33.5, 36.92, 40.38, 43.88, 47.42, 51.0, 54.62, 58.279999999999994, 61.97999999999999, 65.71999999999998, 69.49999999999999, 73.31999999999998, 77.17999999999998, 81.07999999999997, 85.01999999999997, 88.99999999999997, 93.01999999999997, 97.07999999999997, 101.17999999999996, 105.31999999999996, 109.49999999999996, 113.71999999999996, 117.97999999999995, 122.27999999999994, 126.61999999999993, 130.99999999999994]
v leadre info: 31 [15.8, 16.0, 16.2, 16.4, 16.599999999999998, 16.799999999999997, 16.999999999999996, 17.199999999999996, 17.399999999999995, 17.599999999999994, 17.799999999999994, 17.999999999999993, 18.199999999999992,

In [75]:

for i in range(len(t_triger_list)):
    leader_info.append(pybind_idm.SpeedPlannerProfile(
        leadercarlength_list[i], s_leader_list[i], v_leader_list[i], exist_leader_list[i]))
ego_info = pybind_idm.EgoInfo(ego_s,ego_v,ego_a)
idm_params = pybind_idm.IDMParam(
        0.2,  # delta_t
        4.0,  # idm_cosnt_jerk
        3.0,  # delta
        2.0,  # acc_delta
        3.5,  # desired_deceleration
        2.5,  # max_acc 输出限幅
        1.0,  # time_headway
        1.5,  # follow_car_delta
        0.0,  # min_spacing
        3.5,  # min_deceleration 输出限幅
        [9.49, 18.98, 28.48],  # v_std
        [0.88, 1.7, 2.7],     # vel_error_std
        [0.0, 2.78, 11.11, 22.22, 90.0],  # idm_delta_v
        [3.0, 4.0, 5.0, 6.0, 8.0],     # idm_min_spacing_list
        0.0,  # desired_spd
        0.0)  # change_rate

idm_output = pybind_idm.get_idm_output(
        ego_info, leader_info, idm_params, v_refs_list)

idm_output_s_list=[]
idm_output_v_list=[]
idm_output_a_list=[]
idm_output_t_list=[]

for idm_output_list in idm_output:
        idm_output_s_list.append(idm_output_list.s_ref_out)
        idm_output_v_list.append(idm_output_list.v_ref_out)
        idm_output_a_list.append(idm_output_list.a_ref_out)
        idm_output_t_list.append(idm_output_list.t)

# plot sv leader
fig_leader_v_s = figure(width=600, height=400, x_range=[0, 150], y_range=[-5, 30])
fig_leader_v_s.scatter(
        s_leader_list, v_leader_list, color='blue', line_alpha=0.4, legend_label="leader_v_s")
fig_leader_v_s.scatter(
            idm_output_s_list, idm_output_v_list, color='black', line_alpha=0.4, legend_label="ego_idm_v_s")
fig_leader_v_s.xaxis.axis_label = "s/m"
fig_leader_v_s.yaxis.axis_label = "v/m/s"


        
output_notebook()
show(fig_leader_v_s)



Loading BokehJS ...

In [76]:
fig_idm_a_t = figure(width=400, height=200, y_range=[-3.5, 3.5])
fig_idm_a_t.scatter(idm_output_t_list, idm_output_a_list, color='red',
                        line_alpha=0.4, legend_label="idm_a_t")
fig_idm_a_t.xaxis.axis_label = "t/s"
fig_idm_a_t.yaxis.axis_label = "a/mps2"
output_notebook()
show(fig_idm_a_t)


Loading BokehJS ...

In [63]:
idm_output = []
ego_info = pybind_idm.EgoInfo(-1.71109, 14.7982, 0.0606892) # ego_s, ego_v, ego_a
# leadercarlength;  s_leader; s_ref_follow; v_leader; v_ref_follow; a_ref_follow; exist_leader;
SpeedPlannerProfile_0 = pybind_idm.SpeedPlannerProfile(17.3299, 21.6249 , 15.1468, 1)
SpeedPlannerProfile_1 = pybind_idm.SpeedPlannerProfile(17.3299, 24.6555 , 15.1457,  1)
SpeedPlannerProfile_2 = pybind_idm.SpeedPlannerProfile(17.3299, 27.6862 , 15.1446,  1)
SpeedPlannerProfile_3 = pybind_idm.SpeedPlannerProfile(17.3299, 30.7096 , 15.1402,  1)
SpeedPlannerProfile_4 = pybind_idm.SpeedPlannerProfile(17.3299, 33.7257 , 15.1324,  1)
SpeedPlannerProfile_5 = pybind_idm.SpeedPlannerProfile(17.3299, 36.7417 , 15.1246,  1)
SpeedPlannerProfile_6 = pybind_idm.SpeedPlannerProfile(17.3299, 39.777 ,15.131,  1)
SpeedPlannerProfile_7 = pybind_idm.SpeedPlannerProfile(17.3299, 42.8122 , 15.1373,  1)
SpeedPlannerProfile_8 = pybind_idm.SpeedPlannerProfile(17.3299, 45.8485 , 15.1417,  1)
SpeedPlannerProfile_9 = pybind_idm.SpeedPlannerProfile(17.3299, 48.8857 , 15.1442, 1)
SpeedPlannerProfile_10 = pybind_idm.SpeedPlannerProfile(17.3299, 51.923 , 15.1467, 1)
SpeedPlannerProfile_11 = pybind_idm.SpeedPlannerProfile(17.3299, 54.952 , 15.1468,  1)
SpeedPlannerProfile_12 = pybind_idm.SpeedPlannerProfile(17.3299, 57.981 , 15.1468, 1)
SpeedPlannerProfile_13 = pybind_idm.SpeedPlannerProfile(17.3299, 61.0104 , 15.1468, 1)
SpeedPlannerProfile_14 = pybind_idm.SpeedPlannerProfile(17.3299, 64.0402 , 15.1467,  1)
SpeedPlannerProfile_15 = pybind_idm.SpeedPlannerProfile(17.3299, 67.0701 , 15.1467,  1)
SpeedPlannerProfile_16 = pybind_idm.SpeedPlannerProfile(17.3299, 70.0994 , 15.1467,  1)
SpeedPlannerProfile_17 = pybind_idm.SpeedPlannerProfile(17.3299, 73.1288 , 15.1467,  1)
SpeedPlannerProfile_18 = pybind_idm.SpeedPlannerProfile(17.3299, 76.1582 , 15.1467, 1)
SpeedPlannerProfile_19 = pybind_idm.SpeedPlannerProfile(17.3299, 79.1877 , 15.1467,  1)
SpeedPlannerProfile_20 = pybind_idm.SpeedPlannerProfile(17.3299, 82.2172 , 15.1467,  1)
SpeedPlannerProfile_21 = pybind_idm.SpeedPlannerProfile(17.3299, 85.2466 , 15.1467,  1)
SpeedPlannerProfile_22 = pybind_idm.SpeedPlannerProfile(17.3299, 88.276 , 15.1467, 1)
SpeedPlannerProfile_23 = pybind_idm.SpeedPlannerProfile(17.3299, 91.3052 , 15.1467,  1)
SpeedPlannerProfile_24 = pybind_idm.SpeedPlannerProfile(17.3299, 94.3342 , 15.1467,  1)
SpeedPlannerProfile_25 = pybind_idm.SpeedPlannerProfile(17.3299, 97.3632 , 15.1467,  1)
SpeedPlannerProfile_26 = pybind_idm.SpeedPlannerProfile(17.3299, 100.393 , 15.1467,  1)
SpeedPlannerProfile_27 = pybind_idm.SpeedPlannerProfile(17.3299, 103.422 , 15.1467,  1)
SpeedPlannerProfile_28 = pybind_idm.SpeedPlannerProfile(17.3299, 106.452 , 15.1467,  1)
SpeedPlannerProfile_29 = pybind_idm.SpeedPlannerProfile(17.3299, 109.481 , 15.1467, 1)
SpeedPlannerProfile_30 = pybind_idm.SpeedPlannerProfile(5, 500 , 1.79769e+308, 0)

leader_info = [SpeedPlannerProfile_0,
               SpeedPlannerProfile_1,
               SpeedPlannerProfile_2,
               SpeedPlannerProfile_3,
               SpeedPlannerProfile_4,
               SpeedPlannerProfile_5,
               SpeedPlannerProfile_6,
               SpeedPlannerProfile_7,
               SpeedPlannerProfile_8,
               SpeedPlannerProfile_9,
               SpeedPlannerProfile_10,
               SpeedPlannerProfile_11,
               SpeedPlannerProfile_12,
               SpeedPlannerProfile_13,
               SpeedPlannerProfile_14,
               SpeedPlannerProfile_15,
               SpeedPlannerProfile_16,
               SpeedPlannerProfile_17,
               SpeedPlannerProfile_18,
               SpeedPlannerProfile_19,
               SpeedPlannerProfile_20,
               SpeedPlannerProfile_21,
               SpeedPlannerProfile_22,
               SpeedPlannerProfile_23,
               SpeedPlannerProfile_24,
               SpeedPlannerProfile_25,
               SpeedPlannerProfile_26,
               SpeedPlannerProfile_27,
               SpeedPlannerProfile_28,
               SpeedPlannerProfile_29,
               SpeedPlannerProfile_30]

idm_params = pybind_idm.IDMParam(
    0.2, #delta_t
    4.0, # idm_cosnt_jerk
    3.0, # delta
    2.0, # acc_delta
    3.5, # desired_deceleration
    2.5, # max_acc
    1.0, # time_headway
    1.5, # follow_car_delta
    0.0, # min_spacing
    3.5, # min_deceleration
    [9.49, 18.98, 28.48], # v_std
    [0.88, 1.7, 2.7],     # vel_error_std
    [0.0, 2.78, 11.11, 22.22, 90.0], # idm_delta_v
    [3.0, 4.0, 5.0, 6.0, 8.0],     # idm_min_spacing_list
    0.0,  # desired_spd
    0.0)  # change_rate 

v_refs = [22.22] * 31  # 生成包含 31 个 22.22 的列表

idm_output = pybind_idm.get_idm_output(ego_info, leader_info, idm_params, v_refs)

print(idm_output)

for idm_output_object in idm_output:
    print("t:", idm_output_object.t)
    print("s_ref_out:", idm_output_object.s_ref_out)
    print("v_ref_out:", idm_output_object.v_ref_out)
    print("a_ref_out:", idm_output_object.a_ref_out)




[<pybind_idm.IDMOutput object at 0x7f26f7a5b030>, <pybind_idm.IDMOutput object at 0x7f26f7a5b110>, <pybind_idm.IDMOutput object at 0x7f26f7a5b0a0>, <pybind_idm.IDMOutput object at 0x7f26f7a5b0d8>, <pybind_idm.IDMOutput object at 0x7f26f7a5b148>, <pybind_idm.IDMOutput object at 0x7f26f7a5b1b8>, <pybind_idm.IDMOutput object at 0x7f26f7a5b180>, <pybind_idm.IDMOutput object at 0x7f26f7a5b1f0>, <pybind_idm.IDMOutput object at 0x7f26f7a5b260>, <pybind_idm.IDMOutput object at 0x7f26f7a5b228>, <pybind_idm.IDMOutput object at 0x7f26f7a5b298>, <pybind_idm.IDMOutput object at 0x7f26f7a5b308>, <pybind_idm.IDMOutput object at 0x7f26f7a5b2d0>, <pybind_idm.IDMOutput object at 0x7f26f7a5b340>, <pybind_idm.IDMOutput object at 0x7f26f7a5b378>, <pybind_idm.IDMOutput object at 0x7f26f7a5b3e8>, <pybind_idm.IDMOutput object at 0x7f26f7a5ba40>, <pybind_idm.IDMOutput object at 0x7f26f7a5b3b0>, <pybind_idm.IDMOutput object at 0x7f26f7a5bc70>, <pybind_idm.IDMOutput object at 0x7f26f7a5b570>, <pybind_idm.IDMOutp

In [64]:
case_path = './idm_case_file'
case_manager = IDMCaseManager(case_path)

print("get case_1")
case_1 = case_manager.get_case("case_1")

print(case_1)

****************************** IDMCaseManager start read case ******************************
get case_1
no such case case_1
None


In [65]:
case_path = './json_read_test'
case_manager = IDMCaseManager(case_path)

print("get case_1")
case_1 = case_manager.get_case("case_1")

print(case_1)

****************************** IDMCaseManager start read case ******************************


FileNotFoundError: [Errno 2] No such file or directory: '/home/sensetime/ws/common_math/algorithm/idm/./json_read_test'

In [ ]:
case_ploter = IDMCasePloter()

case_1_layout = case_ploter.plot_idm_case(case_1)
output_notebook()
show(case_1_layout)

NameError: name 'case_1' is not defined